In [ ]:
pip install langchain openai beautifulsoup4 requests pinecone-client flask


In [ ]:
from langchain import OpenAI, LLMChain
from langchain.prompts import PromptTemplate
import openai

openai.api_key = "your-openai-api-key"

# Initialize OpenAI LLM
llm = OpenAI(temperature=0.7)

# Create a LangChain instance
prompt = PromptTemplate(input_variables=["topic"], template="Please summarize key points about {topic}.")
chain = LLMChain(llm=llm, prompt=prompt)


In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_articles(topic):
    search_url = f"https://www.google.com/search?q={topic.replace(' ', '+')}+article"
    headers = {"User-Agent": "Mozilla/5.0"}
    response = requests.get(search_url, headers=headers)

    soup = BeautifulSoup(response.text, "html.parser")
    articles = []

    for link in soup.find_all("a"):
        url = link.get("href")
        if "http" in url:
            articles.append(url)

    return articles[:5]  # Return top 5 articles


In [ ]:
def summarize_article(url, chain):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    text = ' '.join([p.text for p in soup.find_all("p")])

    summary = chain.run(topic=text)
    return summary


In [ ]:
import pinecone

pinecone.init(api_key="your-pinecone-api-key", environment="us-west1-gcp")

index = pinecone.Index("research-summaries")

def store_summary(summary, topic):
    index.upsert([(topic, summary)])


In [ ]:
def answer_question(question, index):
    results = index.query(question, top_k=1)
    return results['matches'][0]['metadata']['summary']

question_prompt = PromptTemplate(input_variables=["question"], template="Based on your knowledge, answer the following question: {question}")
question_chain = LLMChain(llm=llm, prompt=question_prompt)

def generate_answer(question, chain, index):
    summary = answer_question(question, index)
    answer = chain.run(question=summary)
    return answer


In [ ]:
def main():
    topic = input("Enter a research topic: ")
    articles = scrape_articles(topic)

    for url in articles:
        summary = summarize_article(url, chain)
        store_summary(summary, topic)

    while True:
        question = input("\nAsk a question or type 'exit': ")
        if question.lower() == 'exit':
            break
        answer = generate_answer(question, question_chain, index)
        print(f"Answer: {answer}")

if __name__ == "__main__":
    main()


In [ ]:
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route("/", methods=["GET", "POST"])
def index():
    if request.method == "POST":
        topic = request.form.get("topic")
        articles = scrape_articles(topic)

        for url in articles:
            summary = summarize_article(url, chain)
            store_summary(summary, topic)

        return render_template("results.html", summaries=summaries)

    return render_template("index.html")

@app.route("/ask", methods=["POST"])
def ask():
    question = request.form.get("question")
    answer = generate_answer(question, question_chain, index)
    return render_template("answer.html", answer=answer)

if __name__ == "__main__":
    app.run(debug=True)
